In [1]:
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, ToolMessage
from langchain_ollama.chat_models import ChatOllama
from langchain_core.messages import HumanMessage
from tqdm import tqdm
import time, re

llm = ChatOllama(
    model="llama3.1:latest",
    temperature=0.0,
    base_url="http://localhost:11434",
    api_key="ollama"
)

In [4]:
@tool
def add(a: int, b: int) -> int:
    """
    Add a and b.
    
    Args:
        a (int): first integer to be added
        b (int): second integer to be added

    Return:
        int: sum of a and b
    """
    return a + b

@tool
def subtract(a: int, b:int) -> int:
    """Subtract b from a."""
    return a - b

@tool
def multiply(a: int, b:int) -> int:
    """Multiply a and b."""
    return a * b

In [15]:
tool_map = {
    "add": add, 
    "subtract": subtract,
    "multiply": multiply
}


In [5]:
tools = [add, subtract, multiply]

llm_with_tools = llm.bind_tools(tools)

In [9]:
query = "What is 3 + 2?"
chat_history = [HumanMessage(content=query)]
response_1 = llm_with_tools.invoke(chat_history)
chat_history.append(response_1)
print(type(response_1))
print(response_1.content)


<class 'langchain_core.messages.ai.AIMessage'>



In [11]:
response_1.tool_calls

[{'name': 'add',
  'args': {'a': 3, 'b': 2},
  'id': '8a91024a-5df1-4de0-a2fa-8f0987a28a56',
  'type': 'tool_call'}]

In [13]:
tool_calls_1 = response_1.tool_calls

tool_1_name = tool_calls_1[0]["name"]
tool_1_args = tool_calls_1[0]["args"]
tool_call_1_id = tool_calls_1[0]["id"]

print(f'tool name:\n{tool_1_name}')
print(f'tool args:\n{tool_1_args}')
print(f'tool call ID:\n{tool_call_1_id}')

tool name:
add
tool args:
{'a': 3, 'b': 2}
tool call ID:
8a91024a-5df1-4de0-a2fa-8f0987a28a56


In [20]:
tool_response = tool_map[tool_1_name].invoke(tool_1_args)
tool_response

5

In [19]:
tool_message = ToolMessage(content=tool_response, tool_call_id=tool_call_1_id)
print(tool_message)

content='5' tool_call_id='8a91024a-5df1-4de0-a2fa-8f0987a28a56'


In [21]:
chat_history.append(tool_message)
answer = llm_with_tools.invoke(chat_history)
print(type(answer))
print(answer.content)

<class 'langchain_core.messages.ai.AIMessage'>
The result of the calculation 3 + 2 is 5.


## Tool Calling Agent

In [22]:
class ToolCallingAgent:
    def __init__(self, llm):
        self.llm_with_tools = llm.bind_tools(tools)
        self.tool_map = tool_map

    def run(self, query: str) -> str:
        # Step 1: Initial user message
        chat_history = [HumanMessage(content=query)]

        # Step 2: LLM chooses tool
        response = self.llm_with_tools.invoke(chat_history)
        if not response.tool_calls:
            return response.contet # Direct response, no tool needed
        # Step 3: Handle first tool call
        tool_call = response.tool_calls[0]
        tool_name = tool_call["name"]
        tool_args = tool_call["args"]
        tool_call_id = tool_call["id"]

        # Step 4: Call tool manually
        tool_result = self.tool_map[tool_name].invoke(tool_args)

        # Step 5: Send result back to LLM
        tool_message = ToolMessage(content=str(tool_result), tool_call_id=tool_call_id)
        chat_history.extend([response, tool_message])

        # Step 6: Final LLM result
        final_response = self.llm_with_tools.invoke(chat_history)
        return final_response.content

In [25]:
my_agent = ToolCallingAgent(llm)

print(my_agent.run("one plus 2 + 4"))

print(my_agent.run("one - 2 +100"))

print(my_agent.run("three times two"))

The result of the expression is 7.
The result of the expression "one - 2 + 100" is -1.
The result of multiplying 3 by 2 is 6.
